In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import pycountry

In [26]:
# Collect URLs 
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recoveries = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
url_latest = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/02-21-2022.csv'
url_us_medical = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/02-21-2022.csv'
# Read data from URLs
confirmed = pd.read_csv(url_confirmed)
deaths = pd.read_csv(url_deaths)
recoveries = pd.read_csv(url_recoveries)
latest = pd.read_csv(url_latest)
us_medical = pd.read_csv(url_us_medical)
# Print data
confirmed.head()
deaths.head()
recoveries.head()
latest.head()
us_medical.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,Date,People_Tested,Mortality_Rate
0,Alabama,US,2022-02-22 04:31:49,32.3182,-86.9023,1273484,17933,NaN,NaN,1.0,...,NaN,NaN,1.408184,84000001.0,USA,NaN,NaN,2022-02-21,NaN,NaN
1,Alaska,US,2022-02-22 04:31:49,61.3707,-152.4044,235956,1141,NaN,NaN,2.0,...,3741734.0,NaN,0.483565,84000002.0,USA,511483.777485,NaN,2022-02-21,NaN,NaN
2,American Samoa,US,2022-02-22 04:31:49,-14.2710,-170.1320,18,0,NaN,NaN,60.0,...,2140.0,NaN,0.000000,16.0,ASM,3846.084722,NaN,2022-02-21,NaN,NaN
3,Arizona,US,2022-02-22 04:31:49,33.7298,-111.4312,1967704,27622,NaN,NaN,4.0,...,18125642.0,NaN,1.403768,84000004.0,USA,249022.485694,NaN,2022-02-21,NaN,NaN
4,Arkansas,US,2022-02-22 04:31:49,34.9697,-92.3731,815299,10292,NaN,NaN,5.0,...,4933978.0,NaN,1.262359,84000005.0,USA,163495.641201,NaN,2022-02-21,NaN,NaN


In [27]:

# Lấy danh sách các cột ngày (bỏ qua các cột thông tin địa lý)
date_columns = confirmed.columns[4:]

# Tổng hợp dữ liệu toàn cầu theo từng ngày
df = pd.DataFrame()
df['Date'] = date_columns
df['Confirmed'] = confirmed[date_columns].sum(axis=0).values
df['Deaths'] = deaths[date_columns].sum(axis=0).values
df['Recovered'] = recoveries[date_columns].sum(axis=0).values
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

# Hiển thị bảng dữ liệu
df.reset_index(inplace=True)
df.head()

,index,Date,Confirmed,Deaths,Recovered,Active
0,0,1/22/20,557,17,30,510
1,1,1/23/20,657,18,32,607
2,2,1/24/20,944,26,39,879
3,3,1/25/20,1437,42,42,1353
4,4,1/26/20,2120,56,56,2008


In [28]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

def plot_treemap(case_type):
    latest_date = date_columns[-1]
    previous_date = date_columns[-2]
    if case_type == 'Confirmed':
        data = confirmed.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Confirmed Cases by Country'
    elif case_type == 'Active':
        active = confirmed.groupby('Country/Region')[latest_date].sum() - \
                 deaths.groupby('Country/Region')[latest_date].sum() - \
                 recoveries.groupby('Country/Region')[latest_date].sum()
        data = active.reset_index()
        data.columns = ['Country', 'Value']
        title = 'Active Cases by Country'
    elif case_type == 'Recovered':
        data = recoveries.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Recovered Cases by Country'
    elif case_type == 'Deaths':
        data = deaths.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Deaths by Country'
    elif case_type == 'Daily Increase':
        daily_increase = confirmed.groupby('Country/Region')[latest_date].sum() - \
                         confirmed.groupby('Country/Region')[previous_date].sum()
        data = daily_increase.reset_index()
        data.columns = ['Country', 'Value']
        title = 'Daily Increase by Country'
    else:
        raise ValueError('case_type không hợp lệ!')

    # Tính phần trăm
    total = data['Value'].sum()
    data['Percent'] = (data['Value'] / total * 100).round(2)
    # Tạo nhãn hiển thị trên tree map
    data['Label'] = data['Country'] + '<br>' + data['Value'].map('{:,}'.format) + '<br>' + data['Percent'].map('{:.0f}%'.format)

    fig = px.treemap(
        data,
        path=['Label'],
        values='Value',
        title=title
    )
    fig.show()

case_selector = widgets.ToggleButtons(
    options=['Confirmed', 'Active', 'Recovered', 'Deaths', 'Daily Increase'],
    description='',
    button_style=''
)
display(case_selector)
widgets.interactive(plot_treemap, case_type=case_selector)

ToggleButtons(options=('Confirmed', 'Active', 'Recovered', 'Deaths', 'Daily Increase'), value='Confirmed')

interactive(children=(ToggleButtons(description='case_type', options=('Confirmed', 'Active', 'Recovered', 'Dea…

In [29]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

def plot_map(case_type):
    latest_date = date_columns[-1]
    previous_date = date_columns[-2]
    if case_type == 'Confirmed':
        data = confirmed.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Confirmed Cases by Country'
    elif case_type == 'Active':
        active = confirmed.groupby('Country/Region')[latest_date].sum() - \
                 deaths.groupby('Country/Region')[latest_date].sum() - \
                 recoveries.groupby('Country/Region')[latest_date].sum()
        data = active.reset_index()
        data.columns = ['Country', 'Value']
        title = 'Active Cases by Country'
    elif case_type == 'Recovered':
        data = recoveries.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Recovered Cases by Country'
    elif case_type == 'Deaths':
        data = deaths.groupby('Country/Region')[latest_date].sum().reset_index()
        data.columns = ['Country', 'Value']
        title = 'Deaths by Country'
    elif case_type == 'Daily Increase':
        daily_increase = confirmed.groupby('Country/Region')[latest_date].sum() - \
                         confirmed.groupby('Country/Region')[previous_date].sum()
        data = daily_increase.reset_index()
        data.columns = ['Country', 'Value']
        title = 'Daily Increase by Country'
    else:
        raise ValueError('case_type không hợp lệ!')

    # Đổi tên quốc gia sang mã ISO
    def get_iso3(country_name):
        try:
            return pycountry.countries.lookup(country_name).alpha_3
        except:
            return None
    data['iso_alpha'] = data['Country'].apply(get_iso3)
    data = data.dropna(subset=['iso_alpha'])

    fig = px.choropleth(
        data,
        locations='iso_alpha',
        color='Value',
        hover_name='Country',
        color_continuous_scale='Blues',
        title=title
    )
    fig.show()

case_selector = widgets.ToggleButtons(
    options=['Confirmed', 'Active', 'Recovered', 'Deaths', 'Daily Increase'],
    description='Chọn loại ca:',
    button_style=''
)
widgets.interactive(plot_map, case_type=case_selector)

interactive(children=(ToggleButtons(description='Chọn loại ca:', options=('Confirmed', 'Active', 'Recovered', …